In [ ]:
# ANÁLISIS EXPLORATORIO DE DATOS - NECROPSIAS GUATEMALA (INE)
# Universidad del Valle de Guatemala - CC3074 Minería de Datos

## 1. CONFIGURACIÓN INICIAL

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pyreadstat
import warnings
import os
import re
from scipy import stats
from scipy.stats import shapiro, normaltest, chi2_contingency
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score, silhouette_samples
from sklearn.decomposition import PCA
import glob

warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")

os.makedirs('resultados', exist_ok=True)
os.makedirs('graficos', exist_ok=True)

print(" Librerías cargadas exitosamente")

## 2. CARGA Y UNIÓN DE DATOS

In [ ]:
# Usar directorio actual
ruta_datos = "."

archivos = glob.glob(os.path.join(ruta_datos, "necropsias*.sav"))
print(f"\n Archivos encontrados: {len(archivos)}")
for archivo in sorted(archivos):
    print(f"   - {os.path.basename(archivo)}")

In [ ]:
def cargar_necropsia(archivo):
    """Carga un archivo .sav de necropsias y agrega columna de año"""
    nombre_archivo = os.path.basename(archivo)
    anio = re.search(r'(\d{4})', nombre_archivo)
    
    if anio:
        anio = int(anio.group(1))
    else:
        print(f"No se pudo extraer el año de: {nombre_archivo}")
        return None
    
    try:
        df, meta = pyreadstat.read_sav(archivo)
        df['anio'] = anio
        print(f"{nombre_archivo}: {len(df)} registros")
        return df
    except Exception as e:
        print(f"Error al cargar {nombre_archivo}: {str(e)}")
        return None

lista_dataframes = []
for archivo in sorted(archivos):
    df = cargar_necropsia(archivo)
    if df is not None:
        lista_dataframes.append(df)

print("\n Uniendo todos los datasets...")
necropsias = pd.concat(lista_dataframes, ignore_index=True, sort=False)

print(f" DATOS CARGADOS EXITOSAMENTE")
print(f"\n Total de registros: {len(necropsias):,}")
print(f" Total de variables: {len(necropsias.columns)}")
print(f" Años disponibles: {sorted(necropsias['anio'].unique())}")

## 3. LIMPIEZA Y CONSOLIDACIÓN DE DATOS

Los archivos de diferentes años tienen nombres de variables ligeramente distintos.
Consolidamos las variables duplicadas para tener un dataset uniforme.

In [ ]:
print("3. LIMPIEZA Y CONSOLIDACIÓN DE DATOS")

# 3.1 Consolidar variables de EDAD
print("\n Consolidando variable EDAD...")
necropsias['edad'] = necropsias['edad_per'].fillna(necropsias['edad_person'])
# Limpiar valores inválidos (999 = desconocido según INE)
necropsias.loc[necropsias['edad'] >= 999, 'edad'] = np.nan
print(f"    edad: {necropsias['edad'].notna().sum():,} valores válidos")

# 3.2 Consolidar variables de SEXO
print("\n Consolidando variable SEXO...")
necropsias['sexo'] = necropsias['sexo_per'].fillna(
    necropsias['sexo_person'].fillna(necropsias['sexo_per_eva'])
)
# Codificar: 1=Hombre, 2=Mujer, 9=No especificado
necropsias.loc[necropsias['sexo'] == 9, 'sexo'] = np.nan
print(f"    sexo: {necropsias['sexo'].notna().sum():,} valores válidos")

# 3.3 Consolidar MES de ocurrencia
print("\n Consolidando variable MES...")
necropsias['mes'] = necropsias['mes_ocu'].fillna(necropsias['mes_ing'])
print(f"    mes: {necropsias['mes'].notna().sum():,} valores válidos")

# 3.4 Consolidar DÍA DE LA SEMANA
print("\n Consolidando variable DÍA DE LA SEMANA...")
necropsias['dia_semana'] = necropsias['diasem_ocu'].fillna(
    necropsias['día_sem_ocu'].fillna(necropsias['dia_sem_ocu'])
)
print(f"    dia_semana: {necropsias['dia_semana'].notna().sum():,} valores válidos")

# 3.5 Consolidar CAUSA DE MUERTE
print("\n Consolidando variable CAUSA DE MUERTE...")
necropsias['causa'] = necropsias['causa_muerte'].fillna(necropsias['eva_mn'])
print(f"    causa: {necropsias['causa'].notna().sum():,} valores válidos")

# 3.6 Consolidar MUNICIPIO
print("\n Consolidando variable MUNICIPIO...")
necropsias['municipio'] = necropsias['mupio_ocu']
print(f"    municipio: {necropsias['municipio'].notna().sum():,} valores válidos")

# 3.7 Consolidar DEPARTAMENTO
necropsias['departamento'] = necropsias['depto_ocu']

# Crear dataset limpio con variables consolidadas
vars_limpias = ['anio', 'mes', 'dia_semana', 'departamento', 'municipio', 
                'edad', 'sexo', 'causa']
df_limpio = necropsias[vars_limpias].copy()

print(f"\n Dataset limpio creado con {len(vars_limpias)} variables consolidadas")
print(f" Registros totales: {len(df_limpio):,}")

# Resumen de valores faltantes después de limpieza
print("\n Valores faltantes después de consolidación:")
for col in vars_limpias:
    faltantes = df_limpio[col].isna().sum()
    pct = faltantes / len(df_limpio) * 100
    print(f"   {col:15s}: {faltantes:,} ({pct:.1f}%)")

## 4. DESCRIPCIÓN DEL DATASET

In [ ]:
print("4. DESCRIPCIÓN DEL DATASET")

print(f"\n Dimensiones del dataset limpio:")
print(f"   - Observaciones (filas): {df_limpio.shape[0]:,}")
print(f"   - Variables (columnas): {df_limpio.shape[1]}")

print(f"\n Variables y tipos:")
for col in df_limpio.columns:
    tipo = str(df_limpio[col].dtype)
    no_nulos = df_limpio[col].notna().sum()
    print(f"   {col:15s} | {tipo:10s} | {no_nulos:,} valores")

In [ ]:
# Estadísticas descriptivas
print("\n Estadísticas descriptivas:")
print(df_limpio.describe())

# Guardar
df_limpio.describe().to_csv('resultados/estadisticas_descriptivas.csv')